# 4.1 데이터에서 학습한다!

데이터에서 학습한다 : 가중치 매개변수의 값을 데이터를 보고 자동으로 결정한다

## 4.1.1 데이터 주도 학습

이미지에서 특징을 추출하고 그 특징의 패턴을 기계학습 기술로 학습하는 방법이 있다.


## 4.1.2

* 기계학습 문제 : 데이터를 훈련 데이터와 시험 데이터로 나눠 학습과 실험을 수행.

   1. 훈련 데이터만 사용하여 학습하면서 최적의 매개변수를 찾는다.

   2. 시험 데이터를 사용하여 앞서 훈련한 모델의 실력을 평가


* 훈련 데이터와 시험 데이터를 나누는 이유?

  우리는 범용적으로 사용할 수 있는 모델을 원하기 때문.

* 범용 능력 : 아직 보지 못한 데이터로도 문제를 올바르게 풀어내는 능력

  -> 범용 능력을 획득하는 것이 기계학습의 최종 목표

* 오버피팅 : 한 데이터셋에만 지나치게 최적화된 상태

  -> 오버피팅 피하기는 기계학습의 중요한 과제
  
# 4.2 손실함수
  
손실함수를 기준으로 최적의 매개변수 값을 탐색

손실함수로는 일반적으로 평균 제곱 오차와 교차 엔트로피 오차를 사용.


## 4.2.1 평균 제곱 오차

$$E=\frac{1}{2}\sum_{k}(y_{k}-t_{k})^{2}$$

In [2]:
import numpy as np
def mean_squared_error(y,t):
    return 0.5*np.sum((y-t)**2)

In [3]:
t=[0,0,1,0,0,0,0,0,0,0]

ex) '2'일 확률이 가장 높다고 추정(0.6)

In [4]:
y=[0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]
mean_squared_error(np.array(y), np.array(t))

0.09750000000000003

ex)'7'일 확률이 가장 높다고 추정(0.6)

In [5]:
y=[0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0]
mean_squared_error(np.array(y), np.array(t))

0.5975

평균 제곱 오차를 기준으로 첫번째 예의 결과가 오차가 더 작으므로 정답에 더 가까울 것이다.
## 4.4.2 교차 엔트로피 오차

$$E=-\sum_{k}t_{k}logy_{k}$$

ex) '2'가 정답이라하고 신경말 출력이 0.6이라면 교차 엔트로피 오차는 -log0.6=0.51이다.

   같은 조건에서 신경망 출력이 0.1 이라면 -log0.1=2.3이다.

In [8]:
def cross_entropy_error(y,t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))

t=[0,0,1,0,0,0,0,0,0,0]

ex) 정답일 때의 출력이 0.6인 경우

In [9]:
y=[0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]
cross_entropy_error(np.array(y),np.array(t))

0.510825457099338

ex) 정답일 때의 출력이 0.1인 경우

In [10]:
y=[0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0]
cross_entropy_error(np.array(y),np.array(t))

2.302584092994546

따라서 오차 값이 더 작은 첫 번째 추정이 정답일 가능성이 높다.
## 4.2.3 미니배치 학습

훈련 데이터 모두에 대한 손살함수의 합을 구하는 방법 : 교차 엔트로피 오차
$$E=-\frac{1}{N}\sum_{n}\sum_{k}t_{nk}logy_{nk}$$

미니배치 학습 : 60,000장의 훈련 데이터 중에서 100장을 무작위로 뽑아 그 100장만을 사용하여 학습하는 것

* MNIST 데이터셋 읽어오는 코드

In [14]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape)

Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!
(60000, 784)
(60000, 10)


* 이 훈련 데이터에서 무작위로 10장만 빼내는 코드

In [17]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch=x_train[batch_mask]
t_batch=t_train[batch_mask]

np.random.choice(60000,10)

array([52857, 49284, 49497, 54268, 12124, 13076, 57590, 13783, 28679,
       34558])

## 4.2.4 (배치용)교차 엔트로피 오차 구현하기

* 미니배치 같은 배치 데이터를 지원하는 교차 엔트로피 오차 구하는 방법

In [19]:
def cross_entropy_error(y,t):
    if y.ndim==1:
        t=t.reshape(1, t.size)
        y=y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y+1e-7)) / batch_size

* 원-핫 인코딩이 아니라 '2'나'7' 등의 숫자 레이블로 주어졌을 때의 교차 엔트로피 오차

In [20]:
def cross_entropy_error(y,t):
    if y.ndim==1:
        t=t.reshape(1, t.size)
        y =y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size),t]+1e-7)) / batch_size

In [25]:
batch_size

5

## 4.2.5 왜 손실 함수를 설정하는가?

-> 높은 정확도를 끌어내는 매개변수의 값을 찾기 위해

* 정확도라는 지표를 두고 손실 함수의 값이라는 우회적인 방법을 택하는 이유?

-> 미분에 주목하면 해결

손실함수의 미분 : '가중치 매개변수의 값을 아주 조금 변화시켰을 때, 손실 함수가 어떻게 변하나'라는 의미

만약 이분 값이 음수면 그 가중치 매개변수를 양의 방향으로 변화시켜 손실 함수의 값을 줄일 수 있다.

하지만 미분 값이 0이면 가중치 매개변수를 어느 쪽으로 움직여도 손실 함수의 값은 줄어들지 않는다.

따라서 정확도를 지표로 삼아서는 안 되는 이유는 미분 값이 대부분의 장소에서 0이 되어 매개변수를 갱신할 수 없기 때문이다.

* 정확도를 지표로 삼으면 매개변수의 미분이 대부분 0이 되는 이유?

예를 들어 100장의 훈련 데이터 중 32장을 올바르게 인식하면 정확도는 32%이다. 만약 정확도가 지표였다면 가중치 매개변수의 값을 조금 바꿔도 정확도는 그대로 32%이다. 즉, 매개변수를 약간만 조정해서는 정확도가 개선되지 않고, 변하더라도 연속적인 변화보다는 불연속적인 뛰엄뛰엄한 값으로 바뀐다.

* 손실 함수를 지표로 삼으면?

매개변수의 값이 조금 변하면 손실 함수의 값도 연속적으로 변한다.

* 계단함수를 활성화 함수로 사용하지 않는 이유?

정확도와 마찬가지로 미소한 변화에는 거의 반응을 보이지 않고, 반응이 있더라도 그 값이 불연속적으로 변화하기 때문이다.

따라서 연속적으로 변하고 곡선의 기울기도 연속으로 변하여 미분의 값이 어느 장소라도 0이 되지 않는 시그모이드 함수를 사용한다.